In [1]:
import torch
import argparse
from transformers import BertModel, BertTokenizer

# Custom imports
from model.utils import *
from model.data_utils import *
from model.transformer import Transformer
from model.bracketing import IdentityChunker, NNSimilarityChunker, cos
from model.generators import IdentityGenerator
from model.classifiers import AttentionClassifier, SeqPairAttentionClassifier
from model.model import MultiTaskNet, End2EndModel


parser = argparse.ArgumentParser(description='Model Options')
#parser.add_argument('integers', metavar='N', type=int, nargs='+',
#                    help='an integer for the accumulator')

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Device being used: {device}")


########### LOAD MODELS AND OPTIMIZER ###########
transformer_net = Transformer(model_class=BertModel,
                              tokenizer_class=BertTokenizer,
                              pre_trained_weights='bert-base-uncased',
                              device=device)

bracketing_net = NNSimilarityChunker(sim_function=cos,
                                     threshold=0.9,
                                     exclude_special_tokens=True,
                                     combinatorics='sequential')

generator_net = IdentityGenerator()

seq_classifier = AttentionClassifier(embedding_dim=768,
                                       sentset_size=2,
                                       batch_size=32,
                                       dropout=0.3,
                                       n_sentiments=4,
                                       pool_mode='concat',
                                       device=device)


seq_pair_classifier = SeqPairAttentionClassifier(embedding_dim=768,
                                              num_classes=4,
                                              batch_size=32,
                                              dropout=0.3,
                                              n_attention_vecs=4,
                                              pool_mode='concat',
                                              device=device)

multitask_net = MultiTaskNet(seq_classifier, 
                             seq_classifier, 
                             device=device)

model = End2EndModel(transformer=transformer_net,
                     bracketer=bracketing_net,
                     generator=generator_net,
                     multitasknet=multitask_net,
                     device=device)

batch_sequence = ['this is one sentence', 'this is a second sentence', 'this is a third sentnen.']


#optimizer = torch.optim.Adam(multitask_net.parameters())



Device being used: cpu


In [3]:
import pandas as pd

# LOAD SST-2
DATA_SST_TRAIN = pd.read_csv('./assets/datasets/SST2/SST2_dataset_train.csv', sep='\t')
DATA_SST_TEST = pd.read_csv('./assets/datasets/SST2/SST2_dataset_test.csv', sep='\t')
DATA_SST_DEV = pd.read_csv('./assets/datasets/SST2/SST2_dataset_dev.csv', sep='\t')

# LOAD QUORA QUESTION PAIRS
DATA_QQP_TRAIN = pd.read_csv('./assets/datasets/QQP/QQP_train.tsv', sep='\t', error_bad_lines=False)
DATA_QQP_TEST = pd.read_csv('./assets/datasets/QQP/QQP_test.tsv', sep='\t', error_bad_lines=False)
DATA_QQP_DEV = pd.read_csv('./assets/datasets/QQP/QQP_dev.tsv', sep='\t', error_bad_lines=False)

b'Skipping line 83032: expected 6 fields, saw 7\n'
b'Skipping line 154657: expected 6 fields, saw 7\n'
b'Skipping line 323916: expected 6 fields, saw 7\n'
/Users/sergicastellasape/miniconda3/envs/za_conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
from model.data_utils import get_batch_SST2_from_indices, get_batch_QQP_from_indices

batch_size = {'SST2': 4, 'QQP': 4}
n_batches = {'SST2': 10, 'QQP': 10}
counter = {'SST2': 0, 'QQP': 0}
get_batch_function = {'SST2': get_batch_SST2_from_indices, 'QQP': get_batch_QQP_from_indices}
dataframe = {'SST2': DATA_SST_TRAIN, 'QQP': DATA_QQP_TRAIN}

datasets = ['SST2', 'QQP']
batch_indices = {}

optimizer = torch.optim.Adam(multitask_net.parameters(), 
                             lr=0.0001, 
                             betas=(0.9, 0.999), 
                             eps=1e-08, 
                             weight_decay=0.01, 
                             amsgrad=False)

global_counter = 0
finished_training = False

while not finished_training:
    for dataset in datasets:
        if counter[dataset] > n_batches[dataset] or global_counter == 0:
            counter[dataset] = 0
            # Re-shuffle the training batches data
            batch_indices[dataset] = torch.randperm(n_batches[dataset]*batch_size[dataset],
                                                    device=device).reshape(-1, batch_size[dataset])
    
    batch_sequences, batch_targets, batch_splits = [], [], [0]
    for dataset in datasets:
        idx = counter[dataset]
        dataset_batch = get_batch_function[dataset](dataframe[dataset], 
                                                    batch_indices[dataset][idx])
        batch_targets.extend([data[1] for data in dataset_batch])
        batch_sequences.extend([data[0] for data in dataset_batch])
        batch_splits.append(batch_splits[-1] + len(dataset_batch))
        counter[dataset] += 1
    
    print(batch_sequences)
    
    model.train()
    batch_predictions = model.forward(batch_sequences, batch_splits=batch_splits)
    L = model.loss(batch_predictions, batch_targets, weights = None)
    L.backward()
    optimizer.step()
    print('Loss: ', L)

["the part where nothing 's happening , ", "are more deeply thought through than in most ` right-thinking ' films ", 'the greatest musicians ', 'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ', ('My Galaxy ace is hang?', 'Why are the people on Staten Island are racist?'), ('How can I make me believe that everything is going good in life and get satisfaction when nothing is going right?', 'What type of government does France currently have and how has it benefited the country?'), ('Where can I find a power outlet for my laptop at Melbourne Airport?', 'Would a second airport in Sydney, Australia be needed if a high-speed rail link was created between Melbourne and Sydney?'), ('How is being gay or lesbian less moral than divorce?', 'Why do a lot of theists and agnostics confuse mainstream atheistic thought with "positive atheism"?')]


AttributeError: 'int' object has no attribute 'size'

In [ ]:

batch_sequence_pairs = [('Why Tamil Nadu Universities are no good ?  lol this is longer.', 'What was the deadliest battle in history?'),
                       ('What are the difference between polyester','What is the difference between cotton and poly'),
                       ('What are the difference between polyester', 'What was the deadliest battle in history?')]
batch_sequence = ['Why Tamil Nadu Universities are no good?', 
                  'What was the deadliest battle in history?',
                  'What are the difference between polyester']
combined = batch_sequence_pairs + batch_sequence
output, masks_dict = transformer_net.forward(combined, return_masks=True)
print(output.size())
print(masks_dict['seq_pair_mask'])

In [ ]:
L = [(1, 3),(1, 3),(6, 0)]
[l[:-1][:] for l in L]

In [ ]:
def function(a, **kwargs):
    print('First argument is: ', a)
    if second_argument:
        print('second argument:', second_argument)
        
function('hoola', second_argument=True)

In [ ]:
LL = [[1, 0, 0, 1], [1, 0], [1, 0, 0, 0, 0, 0, 1]]
dict_L = {}
dict_L['list'] = LL
max_len = max([len(l) for l in dict_L['list']])
print(dict_L['list'])
dict_L['list'] = [L + [1]*(max_len-len(L)) for L in dict_L['list']]
dict_L['list']

In [ ]:
mask1 = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]
special_mask = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
inv = [1-i for i in special_mask]

print(inv)
print(mask1)
out = [a*b for a,b in zip(inv, mask1)]
print(out)
